In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np
with open('ill_url_ids.pickle', 'rb') as file:
    ill_url_ids = pickle.load(file)
def transform(arg, ill_url_ids):
    if arg not in ill_url_ids:
        return arg
    else:
        return ill_url_ids[arg]
alpha = np.array([0.41,0.16,0.105,0.08,0.06,0.05,0.35,0.3,0.25,0.2])
class URL_info():
    def __init__(self):
        self.total_shows = 0
        self.total_clicks = 0
        self.num_first_clicks = 0
        self.num_last_clicks = 0
        self.show_poses = np.zeros((10,),dtype='int')
        self.show_poses_cl = np.zeros((10,),dtype='int')
        self.click_poses = np.zeros((10,),dtype='int')
        self.time = 0
        self.clicks_on_higher_pos = 0
        self.clicks_on_lower_pos = 0
        self.clicks_after_this = 0
        self.clicks_before_this = 0
        self.is_click_on_higher_pos = 0
        self.is_click_on_lower_pos = 0

In [2]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [3]:
all_groups

,QueryId,DocumentId
0,0,1443
1,0,5912
2,0,5963
3,0,6096
4,0,6230
...,...,...
606045,6310,497350
606046,6310,512263
606047,6310,529857
606048,6310,551291


In [4]:
vals = all_groups['DocumentId'].values
qifs = all_groups['QueryId'].values

In [5]:
with open("doc_to_host.pickle", 'rb') as file:
    doc_to_host = pickle.load(file)
with open("QDP.pickle", 'rb') as file:
    url_simple = pickle.load(file)
with open("QHP.pickle", 'rb') as file:
    host_simple = pickle.load(file)

In [6]:
click_feats = dict()
click_feats1 = dict()
click_feats2 = dict()
click_feats['QD_shows'] = []
click_feats['QD_clicks'] = []
click_feats['QD_ctr'] = []

for i in range(1,11):
    click_feats['QD_showpos_{}'.format(i)] = []

for i in range(1,11):
    click_feats['QD_clickpos_{}'.format(i)] = []

for i in range(1,11):
    click_feats1['QD_ctrpos_{}'.format(i)] = []

click_feats1['QD_pbm'] = []
click_feats1['QD_time'] = []
click_feats1['QD_avg_time'] = []

click_feats['QH_shows'] = []
click_feats['QH_clicks'] = []
click_feats['QH_ctr'] = []

for i in range(1,11):
    click_feats2['QH_showpos_{}'.format(i)] = []

for i in range(1,11):
    click_feats2['QH_clickpos_{}'.format(i)] = []

for i in range(1,11):
    click_feats1['QH_ctrpos_{}'.format(i)] = []

click_feats1['QH_pbm'] = []
click_feats1['QH_time'] = []
click_feats1['QH_avg_time'] = []

In [7]:
for i in tqdm(range(len(vals))):
    key = vals[i]
    Q = qifs[i]
    key1 = transform(key, ill_url_ids)
    click_feats['QD_shows'].append(url_simple[Q][key1].total_shows)
    click_feats['QD_clicks'].append(url_simple[Q][key1].total_clicks)
    if url_simple[Q][key1].total_shows==0:
        click_feats['QD_ctr'].append(0.0)
    else:
        click_feats['QD_ctr'].append(url_simple[Q][key1].total_clicks/url_simple[Q][key1].total_shows)
    for j in range(1,11):
        click_feats['QD_showpos_{}'.format(j)].append(url_simple[Q][key1].show_poses[j-1])
    
    for j in range(1,11):
        click_feats['QD_clickpos_{}'.format(j)].append(url_simple[Q][key1].click_poses[j-1])
    
    for j in range(1,11):
        click_feats1['QD_ctrpos_{}'.format(j)].append(url_simple[Q][key1].click_poses[j-1]/(0.0000001+url_simple[Q][key1].show_poses[j-1]))
    
    click_feats1['QD_pbm'].append(url_simple[Q][key1].total_clicks/(np.sum(url_simple[Q][key1].show_poses*alpha)+0.0000001))
    click_feats1['QD_time'].append(url_simple[Q][key1].time)
    click_feats1['QD_avg_time'].append(url_simple[Q][key1].time/(url_simple[Q][key1].total_clicks+0.0000001))

100%|██████████| 606050/606050 [00:54<00:00, 11054.85it/s]


In [8]:
for i in tqdm(range(len(vals))):
    key = vals[i]
    Q = qifs[i]
    key1 = transform(key, ill_url_ids)
    key1 = doc_to_host[key1]
    click_feats['QH_shows'].append(host_simple[Q][key1].total_shows)
    click_feats['QH_clicks'].append(host_simple[Q][key1].total_clicks)
    if host_simple[Q][key1].total_shows==0:
        click_feats['QH_ctr'].append(0.0)
    else:
        click_feats['QH_ctr'].append(host_simple[Q][key1].total_clicks/host_simple[Q][key1].total_shows)
    for j in range(1,11):
        click_feats2['QH_showpos_{}'.format(j)].append(host_simple[Q][key1].show_poses[j-1])
    
    for j in range(1,11):
        click_feats2['QH_clickpos_{}'.format(j)].append(host_simple[Q][key1].click_poses[j-1])
    
    for j in range(1,11):
        click_feats1['QH_ctrpos_{}'.format(j)].append(host_simple[Q][key1].click_poses[j-1]/(0.0000001+host_simple[Q][key1].show_poses[j-1]))
    
    click_feats1['QH_pbm'].append(host_simple[Q][key1].total_clicks/(np.sum(host_simple[Q][key1].show_poses*alpha)+0.0000001))
    click_feats1['QH_time'].append(host_simple[Q][key1].time)
    click_feats1['QH_avg_time'].append(host_simple[Q][key1].time/(host_simple[Q][key1].total_clicks+0.0000001))

100%|██████████| 606050/606050 [00:54<00:00, 11138.94it/s]


In [9]:
with open("click_feats_pair.pickle", 'wb') as file:
    pickle.dump(click_feats, file)

In [10]:
with open("click_feats_pair1.pickle", 'wb') as file:
    pickle.dump(click_feats1, file)

In [11]:
with open("click_feats_pair2.pickle", 'wb') as file:
    pickle.dump(click_feats2, file)

In [12]:
1+2

3